<div,align=right>Lukas Blecher, Christopher Lueken-Winkels</div>
# Exercise 5


In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline

## Constructing the matrix X

In [ ]:
def construct_X(M, alphas, Np = None):
    '''
    M: resulting tomogram size D in one dimension (D=MxM) 
    alphas: list of angles in degrees
    Np: sensor resolution (optional)
    '''
    
    #in case Np is not given we will select one large enough to fit the diagonal of the image beta
    if Np==None:
        Np=np.ceil(np.sqrt(2)*M)
    